In [3]:
import os
from dotenv import load_dotenv
import time 


load_dotenv()

EXECUTION_ENV=os.getenv("DATABRICKS_LOCATION", "dev")
SERVICE_PRINCIPAL_SECRET=os.getenv("SERVICE_PRINCIPAL_SECRET")
SERVICE_PRINCIPAL_ID=os.getenv("SERVICE_PRINCIPAL_ID")
SERVICE_PRINCIPAL_TENANT_ID=os.getenv("SERVICE_PRINCIPAL_TENANT_ID")
STORAGE_ACCOUNT_NAME=os.getenv("STORAGE_ACCOUNT_NAME")
CONTAINER_NAME=os.getenv("CONTAINER_NAME")
CLUSTER_ID=os.getenv("CLUSTER_ID")


if EXECUTION_ENV=="prod":
    
    from databricks.connect import DatabricksSession
    from databricks.sdk import WorkspaceClient
    w = WorkspaceClient()
    dbutils = w.dbutils
    spark = None
    while not spark:
        try:
            spark = DatabricksSession.builder.clusterId(CLUSTER_ID).getOrCreate()
            time.sleep(5)
        except Exception as e:
            print(e)
        finally:
            spark.conf.set(f"fs.azure.account.auth.type.{STORAGE_ACCOUNT_NAME}.dfs.core.windows.net", "OAuth")
            spark.conf.set(f"fs.azure.account.oauth.provider.type.{STORAGE_ACCOUNT_NAME}.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
            spark.conf.set(f"fs.azure.account.oauth2.client.id.{STORAGE_ACCOUNT_NAME}.dfs.core.windows.net", f'{SERVICE_PRINCIPAL_ID}')
            spark.conf.set(f"fs.azure.account.oauth2.client.secret.{STORAGE_ACCOUNT_NAME}.dfs.core.windows.net", f'{SERVICE_PRINCIPAL_SECRET}')
            spark.conf.set(f"fs.azure.account.oauth2.client.endpoint.{STORAGE_ACCOUNT_NAME}.dfs.core.windows.net", f"https://login.microsoftonline.com/{SERVICE_PRINCIPAL_TENANT_ID}/oauth2/token")
elif EXECUTION_ENV=="dev":
    print('hello world')
else:
    raise Exception('no environment set')


hello world
